In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [61]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import dataloader
import matplotlib.pyplot as plt


In [62]:
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split

In [63]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [64]:
import os

print("Train/cat:", os.listdir('/content/animals_classifier/train/cat'))
print("Train/buffalo:", os.listdir('/content/animals_classifier/train/buffalo'))


Train/cat: ['istockphoto-1325997570-612x612.jpg', 'istockphoto-134398339-612x612.jpg', 'istockphoto-104355461-612x612.jpg', 'istockphoto-1361394182-612x612.jpg']
Train/buffalo: ['istockphoto-842356126-612x612.jpg', 'istockphoto-1445453358-612x612.jpg', 'istockphoto-490431110-612x612.jpg', 'istockphoto-842228162-640x640.jpg']


In [65]:
from torchvision import transforms, datasets

In [66]:
import os
def is_valid_image(path):
    return '.ipynb_checkpoints' not in path and path.lower().endswith ((".png", ".jpg", ".jpeg"))

In [67]:
import shutil

# Remove the unwanted hidden folder if it exists
shutil.rmtree('/content/animals_classifier/train/.ipynb_checkpoints', ignore_errors=True)
shutil.rmtree('/content/animals_classifier/val/.ipynb_checkpoints', ignore_errors=True)


In [68]:
from torchvision.datasets import ImageFolder
from torch.utils.data import dataloader

In [69]:
train_data = ImageFolder(
    root='/content/animals_classifier/train',
    transform=transform,
    is_valid_file=is_valid_image
)


In [70]:
val_data = ImageFolder(
    root='/content/animals_classifier/val',
    transform=transform,
    is_valid_file=is_valid_image
)


In [71]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=2, shuffle=False)

print("Classes:", train_data.classes)

Classes: ['buffalo', 'cat']


In [80]:
import torch.nn as nn
import torch.nn.functional as F

class animals_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # 128 → 64
        x = self.pool(F.relu(self.conv2(x)))   # 64 → 32
        x = self.pool(F.relu(self.conv3(x)))   # 32 → 16
        x = x.view(-1, 64 * 16 * 16)           # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [81]:
model = animals_classifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

animals_classifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=16384, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [82]:
from PIL import Image
from torchvision import transforms
import torch

In [84]:

from PIL import Image
import torch
import torchvision.transforms as transforms

def predict_image(image_path, model, class_names):
    model.eval()  # Set to evaluation mode

    # Transform to match training
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        image = image.to(next(model.parameters()).device)
        output = model(image)
        probs = torch.softmax(output, dim=1)[0]  # corrected this line

    # Show class probabilities
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: {probs[i].item():.4f}")

    predicted_index = torch.argmax(probs).item()
    return class_names[predicted_index]


In [87]:
criterion  = nn.CrossEntropyLoss()
optimizer  = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 20                   # tiny dataset → many epochs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        # forward + loss
        logits = model(imgs)
        loss   = criterion(logits, labels)

        # backward + update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}  loss {running_loss:.4f}")


Epoch 1/20  loss 4.0682
Epoch 2/20  loss 3.0653
Epoch 3/20  loss 2.5645
Epoch 4/20  loss 2.3917
Epoch 5/20  loss 2.0215
Epoch 6/20  loss 1.3226
Epoch 7/20  loss 0.5594
Epoch 8/20  loss 0.2771
Epoch 9/20  loss 0.0621
Epoch 10/20  loss 0.0046
Epoch 11/20  loss 0.0015
Epoch 12/20  loss 0.0003
Epoch 13/20  loss 0.0001
Epoch 14/20  loss 0.0000
Epoch 15/20  loss 0.0000
Epoch 16/20  loss 0.0000
Epoch 17/20  loss 0.0000
Epoch 18/20  loss 0.0000
Epoch 19/20  loss 0.0000
Epoch 20/20  loss 0.0000


In [88]:
# 1. Path to an image you want to classify
image_path = '/content/animals_classifier/val/cat/istockphoto-1443562748-612x612.jpg'  # change to buffalo image if needed

# 2. Get the class names from training data
class_names = train_data.classes  # e.g., ['buffalo', 'cat']

# 3. Run the prediction
predicted_class = predict_image(image_path, model, class_names)

# 4. Show the result
print(f"\nPredicted: {predicted_class}")


buffalo: 0.0000
cat: 1.0000

Predicted: cat
